# Maxime Kubryk


# Chalenge 2


## Démarche

### I) Modèle 0 : label majoritaire

L'idée est de prendre les n premiers labels de chaqune des 8 fonctions de score, et de prendre le label majoritaire parmi ces n*8 labels.

Pour choisir n, j'ai testé chaque valeur de n entre 1 et 100 et mesuré la précision obtenue. Cette courbe est dans le notebook **premiers_tests.ipynb**. On constate que le  meilleur score est obtenu pour n=7.

Sur le leader board, ce modèle donne une précision de **~0.970**.

### II) Modèle 1 : prédire les items similaires à la probe

#### a) random forest 

J'ai tenter d'appliquer cette méthode avec des random forests: dans les données train, quand un label est le même que celui de la probe, on met un label 1, et 0 sinon. Les random forests, sont utilisées pour apprendre à placer le bon label dans chaque fonction de score. Ensuite

Une fois le modèle entraîné, les random forest donnent un label 1 (similaire) ou 0 (dissimilaire) à chaque objet retourné par les fonctions de score. Puis on attribue à la sonde le label majoritaire parmis les objets désignés comme similaires à la sonde (labelisés 1).

Pour trouver de bons hyperparamètres aux random forest, j'ai utilisé hyperopt (voir section Random Forest avec hyperopt dans le notebook **Test_arbres.ipynb**. 

Ce modèle donne un score de **~0.971** sur le leaderboard.

#### b) Modèles linéaires et utilisation de kernels approximés

Le but, est toujours de prédire un label 1 pour les items similaires aux probes.

J'ai fait quelques tests dans le notebook **linear_models.ipynb**, en utilisant un noyau approximé (avec RBFSampler ou Nystrom) et un classifieur linéaire entraîné par descente de gradient stochastique.

Ce modèle a l'avantage d'être plus rapide à entraîner mais il donne de moins bons résultats que les random forests.


### III) Modèle 2 : combinaison de rankings via une chaine de Markov

#### a) Markov chain utilisant les rangs

L'idée vient de l'article disponible à l'adresse http://www10.org/cdrom/papers/577/. L'objectif des travaux de ce papier est de trouver un bon algorithme pour combiner plusieurs rankings d'url donnés par plusieurs moteurs de recherche sur internet. Plusieurs algorithmes sont testés, et d'après leur étude, le plus efficace est celui dénommé "MC4", qui repose les chaines de Markov.

Pour cet algorithme, chaque label dans les rankings initiaux est un état de la chaîne de Markov, et les transitions entre états sont définies comme suit :

- *If the current state is page P, then the next state is chosen as follows: first pick a page Q uniformly from the union of all pages ranked by the search engines. If t(Q) < t(P) for a majority of the lists t that ranked both P and Q, then go to Q, else stay in P. *


Pour appliquer cet algorithme, il faut donc identifier les labels présents dans les rankings initiaux, construire la matrice de transition définissant les probabilités de passage d'un état à un autre, puis trouver l'état final le plus probable si on parcours la chaine de Markov à partir de n'importe quel état initial. Pour calculer les probabilités finale de chaque état, on utilise la même méthode que pour l'algorithme du PageRank: on applique récursivement la matrice de transition à un vecteur de probabilité initialement uniforme (une probabilité égale pour chaque état) jusqu'à ce que ce vecteur varie faiblement. En pratique, la convergence est atteinte en 4 ou 5 itérations. 

La construction de la matrice de transition n'est pas détaillée dans l'article, et les auteurs ne font mention des problèmes comme la suppression des "dead ends" ou des "spider traps", qu'il faut adresser pour avoir une chaine de Markov non pathologique. J'ai trouvé plusieurs façons de "compter" les probabilités de transitions et de résoudre les cas pathologiques, qui correspondent aux fonctions *"build_transition1"* à *"build_transition8"* dans le fichier **markov_chains.py**. Des visualisations de ces matrices sont données dans la Section "Tests" du notebook **Markov-chain.ipynb**.

Pour compter les probabilités de passer de l'état P à Q, on peut (avec rang=0 le plus élevé):
- (1) ajouter +1 quand rang(Q)< rang(P), ajouter -1 quand rang(Q)>rang(P) dans chaque ranking où P et Q sont présents.
- (2) ajouter rang(P)-rang(Q) quand rang(Q)< rang(P), ajouter rang(Q)-rang(P) quand rang(Q)>rang(P) dans chaque ranking où P et Q sont présents.

Une fois les comptages faits, on peut 
- (3) les laisser tels quels
- (4) ou bien mettre toutes les probabilités non-nulles à 1. 

Les valeurs négatives sont mises à zéro, et on normalise ensuite les colonnes de la matrice de transition, pour avoir une somme de 1 dans chaque colonne.

Pour enlever les "dead ends" et les "spider traps", on peut:
- (5) ajouter une valeur $\epsilon$ à toute la matrice de transition, i.e. ajouter une probabilité constante de passer de n'importe quel état vers n'importe quel autre état.
- (6) adapter la valeur de $\epsilon$ à chaque colonne de la matrice de transition, en fonction du nombre de transitions non-nulles dans cette colonne.

Il y a donc 8 modèles possibles ($2^3$), en combinant ces différentes options.

J'ai cherché ensuite par cross validation quelle est le modèle le plus efficace, et les résultats sont donnés dans le notebook **Markov-chain.ipynb** à la section *"Test des 8 build_transition"*. Il ressort que la méthode build_transition8 donne la meilleure précision moyenne sur des échantillons différents de 9473 probes. Cette méthode prends les options (2), (4) et (5). 

Par ailleurs, d'autres tests montrent que prendre seulement les 50 premiers résultats de chaque fonctions de scores initales donne de meilleurs résultats qu'en prenant seulement les 7 ou les 15 premiers résultats. Prendre les 100 premiers ne semble pas améliorer le score, on prendra donc les 50 premiers pour réduire le temps de calcul. Et $\epsilon=0.3$ donne de meilleurs résultats que $\epsilon=0.1$ ou $\epsilon=0.01$.

Sur le leaderboard, le modèle prenant 'build_fonstion8' avec $\epsilon=0.3$ sur les 50 premiers résultats des rankings donne un score de **~0.9759**. Ce qui est le meilleur score que j'ai atteint.

Ce modèle appelle la fonction *find_label()* dans **markov-chains.py**, et l'applique à chaque probe.

#### b) Markov chain utilisant les scores

Pour tenter d'améliorer le modèle précédent, j'ai testé en remplaçant les différences de rangs dans l'option (2) par des différences de scores initiaux. Les matrices de transitions de ce modèle sont construite avec les fonctions *"build_transition_with_scores1"* ou *"build_transition_with_scores2"* dans **markov_chains.py**.

Les résultats de ce modèle sont soit équivalents soit moins bons que le modèle précédent.

Ce modèle appelle la fonction *find_label_with_scores()* dans **markov-chains.py**, et l'applique à chaque probe.

### IV) Modèle 3 : combinaison de chaînes de Markov

Pour tenter d'améliorer le modèle précédent, j'ai essayé d'appliquer une même logique que pour améliorer les arbres de décisions avec de la randomisation (les random forests).

L'idée est de produire un nombre N de meta-rankings avec les chaînes de Markov, mais chaque meta-rankings ayant reçu en input, une selection aléatoire de rankings parmis les 8 rankings des données du challenge. Ce modèle donne des résultats similaires au modèle 3.

Pour ajouter de l'aléatoire, le nombre de premiers items à prendre dans les rankings initiaux est aussi déterminé aléatoirement, de même que le paramètre $\epsilon$.  

Les tests avec ce modèle sont dans le notebook **random_rankings_buildF8.ipynb**, et il ne donne pas de meilleurs résultats que le modèle précédent.

Ce modèle appelle la fonction *random_rankings()* dans **markov-chains.py**, et l'applique à chaque probe.



# Code du modèle ayant donné le meilleur score (modèle 2):

In [1]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import itertools
from joblib import Parallel, delayed
import multiprocessing
import operator
import time


%matplotlib inline

In [ ]:
# loading the function
from markov_chains import find_label


# path to the TEST data set
path = '/home/max/projects/challengeMDI343/data/data_test.mat'

# loading data set
train = io.loadmat(path)

# building dictionnary {Ids: labels}
gallery = {train['galleryId'].ravel()[i]: train['galleryLabel'].ravel()[i] for i in range(len(train['galleryLabel']))}


# selecting only the first 50 items of each initial rankings
resultsIds = test['resultsId'][:,:,:50] 
probeIds = test['probeId'][:,0]

t0 = time.time()




# loop on the probes in the training set.
# The function find_label() from markov-chains.py is applied to each probe.
# For each probe, the function returns the Id of the highest item in the markov-chain meta ranking. 

# paralellized loop using the joblib module
num_cores = multiprocessing.cpu_count()
best_Ids = Parallel(n_jobs=num_cores - 2)(delayed(find_label)(resultsIds[i,:,:], probeIds[i], 
                                              eps=3e-1, tol=1e-5, select_model=7) 
                                              for i in range(len(predIds)))

chron = (time.time()-t0)/60
print 'time:', chron , 'min'






# putting the returned results in a better form 
best_Ids = {k.keys()[0]: k.values()[0] for k in best_Ids}

# linking best_ids to their corresponding label
preds = np.asarray([int(gallery[best_Ids[i]]) for i in probeIds])

print preds
print preds.shape

# printting
np.savetxt('soumission.txt', preds, fmt='%i')